In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import csv
import time
import random

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import csv
import time
import random
import threading
from queue import Queue
import concurrent.futures

class NewsCrawler:
    def __init__(self, start_url, max_pages=20000, max_depth=16, num_threads=7):
        self.start_url = start_url
        self.max_pages = max_pages
        self.max_depth = max_depth
        self.num_threads = num_threads
        self.visited = set()
        self.fetch_results = []
        self.visit_results = []
        self.url_results = []
        self.queue = Queue()
        self.lock = threading.Lock()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }

    def is_valid_url(self, url):
        parsed = urlparse(url)
        return 'nytimes.com' in parsed.netloc and parsed.scheme in ('http', 'https')

    def crawl_worker(self):
        while True:
            item = self.queue.get()
            if item is None:
                break
            url, depth = item

            if depth > self.max_depth:
                self.queue.task_done()
                continue

            with self.lock:
                if url in self.visited or len(self.visited) >= self.max_pages:
                    self.queue.task_done()
                    continue
                self.visited.add(url)
                print(f"Visiting: {url} | Depth: {depth}")

            try:
                response = requests.get(url, headers=self.headers, timeout=10)

                # Log the size and first few characters of the page for debugging
                print(f"Response content length: {len(response.content)}")
                print(f"Response content (first 500 chars): {response.text[:500]}")

                with self.lock:
                    self.fetch_results.append((url, str(response.status_code)))

                if response.status_code == 200:
                    content_type = response.headers.get('Content-Type', '').split(';')[0]
                    size = len(response.content)
                    soup = BeautifulSoup(response.text, 'html.parser')
                    links = soup.find_all('a', href=True)
                    outlinks = len(links)

                    with self.lock:
                        self.visit_results.append((url, size, outlinks, content_type))

                    # Log every link found on the page
                    if not links:
                        print("No links found on this page.")
                    for link in links:
                        full_url = urljoin(url, link['href'])
                        print(f"Found URL: {full_url}")  # Log the found URL

                        if self.is_valid_url(full_url):
                            with self.lock:
                                print(f"Queueing URL: {full_url}")
                                self.url_results.append((full_url, 'OK'))
                                self.queue.put((full_url, depth + 1))
                        else:
                            with self.lock:
                                self.url_results.append((full_url, 'N_OK'))

            except Exception as e:
                print(f"Error crawling {url}: {e}")

            time.sleep(random.uniform(1, 3))  # Add random sleep to avoid overloading the server
            self.queue.task_done()

    def crawl(self):
        self.queue.put((self.start_url, 0))

        with concurrent.futures.ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            futures = [executor.submit(self.crawl_worker) for _ in range(self.num_threads)]

            self.queue.join()

            for _ in range(self.num_threads):
                self.queue.put(None)

            for future in futures:
                future.result()

    def save_results(self):
        with open('fetch_news.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['URL', 'Status'])
            writer.writerows(self.fetch_results)

        with open('visit_news.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['URL', 'Size (bytes)', 'Outlinks', 'Content-Type'])
            writer.writerows(self.visit_results)

        with open('urls_news.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['URL', 'URL_Type'])
            writer.writerows(self.url_results)

if __name__ == '__main__':
    start_url = 'https://www.nytimes.com/'  # or use 'https://www.nytimes.com/'
    crawler = NewsCrawler(start_url, num_threads=7)
    crawler.crawl()
    crawler.save_results()
    print(f"Crawling completed. Visited {len(crawler.visited)} pages.")


Streaming output truncated to the last 5000 lines.
Found URL: https://cn.nytimes.com/society/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/society/?utm_source=nav-footer
Found URL: https://cn.nytimes.com/foreign-relations/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/foreign-relations/?utm_source=nav-footer
Found URL: https://cn.nytimes.com/hk-taiwan/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/hk-taiwan/?utm_source=nav-footer
Found URL: https://cn.nytimes.com/business/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/business/?utm_source=nav-footer
Found URL: https://cn.nytimes.com/global-ec/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/global-ec/?utm_source=nav-footer
Found URL: https://cn.nytimes.com/china-ec/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/china-ec/?utm_source=nav-footer
Found URL: https://cn.nytimes.com/dealbook/?utm_source=nav-footer
Queueing URL: https://cn.nytimes.com/dealbook/?utm_so

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Streaming output truncated to the last 5000 lines.
Queueing URL: https://www.nytimes.com/section/business?page=2
Found URL: https://www.nytimes.com/section/business?page=3
Queueing URL: https://www.nytimes.com/section/business?page=3
Found URL: https://www.nytimes.com/section/business?page=4
Queueing URL: https://www.nytimes.com/section/business?page=4
Found URL: https://www.nytimes.com/section/business?page=5
Queueing URL: https://www.nytimes.com/section/business?page=5
Found URL: https://www.nytimes.com/section/business?page=6
Queueing URL: https://www.nytimes.com/section/business?page=6
Found URL: https://www.nytimes.com/section/business?page=7
Queueing URL: https://www.nytimes.com/section/business?page=7
Found URL: https://www.nytimes.com/section/business?page=8
Queueing URL: https://www.nytimes.com/section/business?page=8
Found URL: https://www.nytimes.com/section/business?page=9
Queueing URL: https://www.nytimes.com/section/business?page=6#after-mid7
Found URL: https://www.nytime

In [ ]:
!apt-get update -y
!apt install -y xvfb
!pip install pyvirtualdisplay selenium
!apt install -y chromium-browser

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack